In [6]:
# ==============================
# NOTEBOOK 3 — Job Cleaning
# ==============================
# GOAL:
# Produce a clean jobs file for downstream vectorization (Notebook 4).
# Output: data/processed/job_postings_cleaned.csv


In [28]:
# ==============================
# STEP 0 — Setup & Imports
# ==============================
BASE_DIR = r"D:\Projects\ResumeJobRecommender"
RAW_DIR  = BASE_DIR + r"\data\raw"
PROC_DIR = BASE_DIR + r"\data\processed"

import os, re, unicodedata
import pandas as pd
import numpy as np

os.makedirs(PROC_DIR, exist_ok=True)
pd.set_option("display.max_colwidth", 180)

print("RAW_DIR :", RAW_DIR)
print("PROC_DIR:", PROC_DIR)


RAW_DIR : D:\Projects\ResumeJobRecommender\data\raw
PROC_DIR: D:\Projects\ResumeJobRecommender\data\processed


In [29]:
# ==============================
# STEP 1 — Load raw jobs & basic checks
# ==============================
jobs_path = RAW_DIR + r"\job_postings.csv"
job_raw = pd.read_csv(jobs_path)
print("Raw jobs shape:", job_raw.shape)
print("Raw columns:", job_raw.columns.tolist())

# quick null snapshot (top 15 columns by null count)
nulls = job_raw.isna().sum().sort_values(ascending=False)
print("\nNull counts (top 15):\n", nulls.head(15))

# dtypes snapshot
print("\nDtypes:\n", job_raw.dtypes.head(15))


Raw jobs shape: (15886, 27)
Raw columns: ['job_id', 'company_id', 'title', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'formatted_work_type', 'location', 'applies', 'original_listed_time', 'remote_allowed', 'views', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type']

Null counts (top 15):
 skills_desc                   15742
closed_time                   14958
med_salary                    14905
remote_allowed                13546
max_salary                    10365
min_salary                    10365
compensation_type              9384
currency                       9384
pay_period                     9384
applies                        7186
posting_domain                 6842
application_url                6091
formatted_experience_level     4902
views                          2763
company_i

In [30]:
# ==============================
# STEP 2 — Keep useful columns (text + minimal metadata)
# ==============================
# Prefer formatted_work_type over work_type to avoid collisions.
base_keep = [
    "job_id","title","description","skills_desc","location",
    "formatted_experience_level","formatted_work_type","work_type",
    "min_salary","max_salary","pay_period","currency",
    "remote_allowed","sponsored","job_posting_url"
]

# if formatted_work_type exists, drop raw work_type from keep list
keep_cols = base_keep.copy()
if "formatted_work_type" in job_raw.columns and "work_type" in keep_cols:
    keep_cols.remove("work_type")

keep_cols = [c for c in keep_cols if c in job_raw.columns]
job_df = job_raw[keep_cols].copy()
print("Kept columns:", job_df.columns.tolist())


Kept columns: ['job_id', 'title', 'description', 'skills_desc', 'location', 'formatted_experience_level', 'formatted_work_type', 'min_salary', 'max_salary', 'pay_period', 'currency', 'remote_allowed', 'sponsored', 'job_posting_url']


In [31]:
# ==============================
# STEP 3 — Rename columns to a consistent schema
# ==============================
rename_map = {
    "title": "job_title",
    "description": "job_description",
    "formatted_experience_level": "experience_level",
    "formatted_work_type": "work_type"  # prefer formatted over raw
}
job_df.rename(columns=rename_map, inplace=True)
print("Columns after rename:", job_df.columns.tolist())


Columns after rename: ['job_id', 'job_title', 'job_description', 'skills_desc', 'location', 'experience_level', 'work_type', 'min_salary', 'max_salary', 'pay_period', 'currency', 'remote_allowed', 'sponsored', 'job_posting_url']


In [32]:
# ==============================
# STEP 4 — Append skills_desc to job_description (safe even if sparse)
# ==============================
if "skills_desc" in job_df.columns:
    non_null_skills = job_df["skills_desc"].notna().sum()
    print(f"'skills_desc' non-null rows: {non_null_skills}")
    job_df["job_description"] = job_df["job_description"].fillna("") + " " + job_df["skills_desc"].fillna("")
    job_df.drop(columns=["skills_desc"], inplace=True, errors="ignore")
else:
    print("'skills_desc' not present — skipping merge.")


'skills_desc' non-null rows: 144


In [33]:
# ==============================
# STEP 5 — Handle missing values (drop empties, fill sensible defaults)
# ==============================
# 5.1 drop rows with empty description (cannot vectorize)
before = len(job_df)
job_df = job_df.dropna(subset=["job_description"]).copy()
after = len(job_df)
print(f"Dropped rows with empty description: {before - after}")

# 5.2 fill object columns
obj_defaults = {
    "experience_level": "Not Specified",
    "pay_period": "Not Specified",
    "currency": "Not Specified",
    "location": "Unknown",
    "job_posting_url": "Not Specified"
}
for c, v in obj_defaults.items():
    if c in job_df.columns:
        job_df[c] = job_df[c].fillna(v).astype(str)

# 5.3 numeric: salaries -> float, flags -> int 0/1
for c in ["min_salary","max_salary"]:
    if c in job_df.columns:
        job_df[c] = job_df[c].fillna(0).astype(float)

for c in ["remote_allowed","sponsored"]:
    if c in job_df.columns:
        job_df[c] = job_df[c].fillna(0).astype(int)

print("\nNulls after fill:\n", job_df.isna().sum())


Dropped rows with empty description: 0

Nulls after fill:
 job_id              0
job_title           0
job_description     0
location            0
experience_level    0
work_type           0
min_salary          0
max_salary          0
pay_period          0
currency            0
remote_allowed      0
sponsored           0
job_posting_url     0
dtype: int64


In [34]:
# ==============================
# STEP 6 — Normalize titles and descriptions (mirror resume cleaning)
# ==============================
def clean_job_title(t):
    t = str(t).strip().lower()
    t = re.sub(r"[^\w\s]", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t

job_df["job_title_clean"] = job_df["job_title"].astype(str).apply(clean_job_title)

def clean_text(s):
    if pd.isna(s):
        return ""
    t = str(s)
    # minimal mojibake fixes (seen in public datasets)
    t = t.replace("Ã¢Â€Â¢", "•").replace("Ã¢Â€Â“", "-").replace("Ã¢Â€Â™", "'")
    t = t.replace("NaÃ¯ve", "Naive").replace("Ã©", "e")
    # normalize & strip html/urls/punct
    t = unicodedata.normalize("NFKC", t)
    t = re.sub(r"<[^>]+>", " ", t)
    t = re.sub(r"http\S+|www\S+|https\S+", " ", t)
    t = re.sub(r"[^\w\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    return t

job_df["job_description_clean"] = job_df["job_description"].apply(clean_text)

print("\nPreview (title → clean, desc_clean):")
display(job_df[["job_title","job_title_clean","job_description_clean"]].head(3))



Preview (title → clean, desc_clean):


,job_title,job_title_clean,job_description_clean
0,Licensed Insurance Agent,licensed insurance agent,while many industries were hurt by the last few years people still need insurance this position is an amazing opportunity for someone who wants a career in the insurance indust...
1,Sales Manager,sales manager,are you a dynamic and creative marketing professional looking to make a significant impact in the world of freight and logistics we are seeking a talented marketing manager to ...
2,Model Risk Auditor,model risk auditor,join us as a model risk auditor showcase your financial services auditing expertise are you an experienced financial services auditor with a passion for unraveling complexities...


In [35]:
# ==============================
# STEP 7 — Duplicate analysis (report only, we KEEP them)
# ==============================
# a) exact duplicates in RAW description (excludes first occurrences)
dup_count_raw = job_df["job_description"].duplicated().sum()
# b) exact duplicates in CLEANED description (excludes first occurrences)
dup_count_clean = job_df["job_description_clean"].duplicated().sum()

# c) rows that belong to duplicate sets (mark both sides)
dup_mask_sets = job_df.duplicated(subset=["job_description_clean"], keep=False)
dup_df = job_df[dup_mask_sets].copy()

# d) groups of identical cleaned descriptions
dup_groups = (dup_df.groupby("job_description_clean", as_index=False)
                     .size()
                     .sort_values("size", ascending=False))

print(f"\nExact duplicate descriptions (raw):   {dup_count_raw}")
print(f"Exact duplicate descriptions (clean): {dup_count_clean}")
print(f"Rows in duplicate sets (clean keep=False): {dup_df.shape[0]}")
print(f"Duplicate groups (clean): {dup_groups.shape[0]}")

# show a few samples side-by-side
sample_dups = dup_df.sample(min(5, len(dup_df)), random_state=42)
sample_dups = sample_dups.assign(
    raw_len = sample_dups["job_description"].astype(str).str.len(),
    clean_len = sample_dups["job_description_clean"].astype(str).str.len(),
    raw_snip = sample_dups["job_description"].astype(str).str.slice(0, 200),
    clean_snip = sample_dups["job_description_clean"].astype(str).str.slice(0, 200),
)
display(sample_dups[["job_id","job_title","location","raw_len","clean_len","raw_snip","clean_snip"]])

print("Decision: Keep all postings (duplicates reflect real reposts / multi-location). No dropping performed.")



Exact duplicate descriptions (raw):   2014
Exact duplicate descriptions (clean): 2054
Rows in duplicate sets (clean keep=False): 2829
Duplicate groups (clean): 775


,job_id,job_title,location,raw_len,clean_len,raw_snip,clean_snip
7801,3697354718,Retail Salesperson,"Gastonia, NC",9216,8941,"Company Overview\nBridgestone Retail Operations (BSRO) is part of Bridgestone Americas and employs over 22,000 teammates in North America. BSRO operates more than 2,200 company...",company overview bridgestone retail operations bsro is part of bridgestone americas and employs over 22 000 teammates in north america bsro operates more than 2 200 company own...
9116,3697387500,"Staff Software Engineer, Google Cloud Unified Fulfillment Optimization","Sunnyvale, CA",4163,4031,"Note: By applying to this position you will have an opportunity to share your preferred working location from the following: Sunnyvale, CA, USA; Kirkland, WA, USA; New York, NY...",note by applying to this position you will have an opportunity to share your preferred working location from the following sunnyvale ca usa kirkland wa usa new york ny usa pitt...
15047,3701325298,Sales Director {Owner/Operator},"St Louis, MO",1997,1956,The Entrepreneur we seek is an experienced sales professional and a natural-born closer who lives for the thrill of the hunt and making serious bank. If you want the freedom to...,the entrepreneur we seek is an experienced sales professional and a natural born closer who lives for the thrill of the hunt and making serious bank if you want the freedom to ...
10083,3699057348,Cashier,"Oakland, CA",4492,4341,"Privacy Notice: Visit https://www.raisingcanes.com/candidate-privacy-notice/ to review our Candidate Privacy Notice.\nStarting hiring pay range (based on location, experience, ...",privacy notice visit to review our candidate privacy notice starting hiring pay range based on location experience qualifications etc 15 50 16 50 hour as a cashier at raising c...
2427,3693052369,HazMat Field Driver/Technician - CDL (A),"Tampa, FL",4580,4464,"Republic Services is a leader in Environmental Services, partnering with customers to create a more sustainable world.\nWe are Sustainability in Action — our promise to be envi...",republic services is a leader in environmental services partnering with customers to create a more sustainable world we are sustainability in action our promise to be environme...


✅ Decision: Keep all postings (duplicates reflect real reposts / multi-location). No dropping performed.


In [36]:
# ==============================
# STEP 8 — QA: structure & length checks
# ==============================
print("Final jobs shape:", job_df.shape)
print("\nNulls:\n", job_df.isna().sum())

for col in ["experience_level","work_type","pay_period","currency","location"]:
    if col in job_df.columns:
        print(f"\nValue counts: {col}")
        print(job_df[col].value_counts(dropna=False).head(10))

job_df["desc_len"] = job_df["job_description_clean"].astype(str).str.len()
print("\nDescription length summary:\n", job_df["desc_len"].describe())
job_df.drop(columns=["desc_len"], inplace=True)


Final jobs shape: (15886, 15)

Nulls:
 job_id                   0
job_title                0
job_description          0
location                 0
experience_level         0
work_type                0
min_salary               0
max_salary               0
pay_period               0
currency                 0
remote_allowed           0
sponsored                0
job_posting_url          0
job_title_clean          0
job_description_clean    0
dtype: int64

Value counts: experience_level
experience_level
Mid-Senior level    5083
Not Specified       4902
Entry level         3694
Associate           1220
Director             687
Internship           166
Executive            134
Name: count, dtype: int64

Value counts: work_type
work_type
Full-time     12844
Contract       1739
Part-time      1010
Temporary       121
Internship      111
Other            53
Volunteer         8
Name: count, dtype: int64

Value counts: pay_period
pay_period
Not Specified    9384
YEARLY           3955
HOURLY     

In [37]:
#  check average text length for duplicates
dup_df["desc_len"] = dup_df["job_description_clean"].str.len()
print(dup_df["desc_len"].describe())

# check a few duplicates manually
display(dup_df.sample(5, random_state=42)[["job_id","job_title","location","job_description"]])


count     2829.000000
mean      4115.833864
std       2031.426334
min          0.000000
25%       2540.000000
50%       3944.000000
75%       5412.000000
max      13455.000000
Name: desc_len, dtype: float64


,job_id,job_title,location,job_description
7801,3697354718,Retail Salesperson,"Gastonia, NC","Company Overview\nBridgestone Retail Operations (BSRO) is part of Bridgestone Americas and employs over 22,000 teammates in North America. BSRO operates more than 2,200 company..."
9116,3697387500,"Staff Software Engineer, Google Cloud Unified Fulfillment Optimization","Sunnyvale, CA","Note: By applying to this position you will have an opportunity to share your preferred working location from the following: Sunnyvale, CA, USA; Kirkland, WA, USA; New York, NY..."
15047,3701325298,Sales Director {Owner/Operator},"St Louis, MO",The Entrepreneur we seek is an experienced sales professional and a natural-born closer who lives for the thrill of the hunt and making serious bank. If you want the freedom to...
10083,3699057348,Cashier,"Oakland, CA","Privacy Notice: Visit https://www.raisingcanes.com/candidate-privacy-notice/ to review our Candidate Privacy Notice.\nStarting hiring pay range (based on location, experience, ..."
2427,3693052369,HazMat Field Driver/Technician - CDL (A),"Tampa, FL","Republic Services is a leader in Environmental Services, partnering with customers to create a more sustainable world.\nWe are Sustainability in Action — our promise to be envi..."


In [24]:
# Total rows initially
print("Total rows:", job_df.shape[0])

# Check number of truly identical job descriptions (raw, before cleaning)
dup_raw = job_df["job_description"].duplicated().sum()
print("Exact duplicate descriptions (raw text):", dup_raw)

# Check identical after cleaning
dup_clean = job_df["job_description_clean"].duplicated().sum()
print("Exact duplicate descriptions (clean text):", dup_clean)


Total rows: 15886
Exact duplicate descriptions (raw text): 2014
Exact duplicate descriptions (clean text): 2054


In [25]:
# Compare raw vs clean for same rows
sample_dups = job_df[job_df.duplicated(subset=["job_description_clean"], keep=False)].sample(5, random_state=42)
display(sample_dups[["job_id", "job_title", "job_description", "job_description_clean"]])


,job_id,job_title,job_description,job_description_clean
7801,3697354718,Retail Salesperson,"Company Overview\nBridgestone Retail Operations (BSRO) is part of Bridgestone Americas and employs over 22,000 teammates in North America. BSRO operates more than 2,200 company...",company overview bridgestone retail operations bsro is part of bridgestone americas and employs over 22 000 teammates in north america bsro operates more than 2 200 company own...
9116,3697387500,"Staff Software Engineer, Google Cloud Unified Fulfillment Optimization","Note: By applying to this position you will have an opportunity to share your preferred working location from the following: Sunnyvale, CA, USA; Kirkland, WA, USA; New York, NY...",note by applying to this position you will have an opportunity to share your preferred working location from the following sunnyvale ca usa kirkland wa usa new york ny usa pitt...
15047,3701325298,Sales Director {Owner/Operator},The Entrepreneur we seek is an experienced sales professional and a natural-born closer who lives for the thrill of the hunt and making serious bank. If you want the freedom to...,the entrepreneur we seek is an experienced sales professional and a natural born closer who lives for the thrill of the hunt and making serious bank if you want the freedom to ...
10083,3699057348,Cashier,"Privacy Notice: Visit https://www.raisingcanes.com/candidate-privacy-notice/ to review our Candidate Privacy Notice.\nStarting hiring pay range (based on location, experience, ...",privacy notice visit to review our candidate privacy notice starting hiring pay range based on location experience qualifications etc 15 50 16 50 hour as a cashier at raising c...
2427,3693052369,HazMat Field Driver/Technician - CDL (A),"Republic Services is a leader in Environmental Services, partnering with customers to create a more sustainable world.\nWe are Sustainability in Action — our promise to be envi...",republic services is a leader in environmental services partnering with customers to create a more sustainable world we are sustainability in action our promise to be environme...


In [27]:
dup_df.shape

(2829, 17)

In [48]:
job_df.head(2)

,job_id,job_title,job_description,location,experience_level,work_type,min_salary,max_salary,pay_period,currency,remote_allowed,sponsored,job_posting_url,job_title_clean,job_description_clean
0,85008768,Licensed Insurance Agent,"While many industries were hurt by the last few years, people still need insurance! This position is an amazing opportunity for someone who wants a career in the insurance indu...","Chico, CA",Not Specified,Full-time,45760.0,52000.0,YEARLY,USD,0,1,https://www.linkedin.com/jobs/view/85008768/?trk=jobs_biz_prem_srch,licensed insurance agent,while many industries were hurt by the last few years people still need insurance this position is an amazing opportunity for someone who wants a career in the insurance indust...
1,133114754,Sales Manager,Are you a dynamic and creative marketing professional looking to make a significant impact in the world of freight and logistics? We are seeking a talented Marketing Manager to...,"Santa Clarita, CA",Not Specified,Full-time,0.0,0.0,Not Specified,Not Specified,0,0,https://www.linkedin.com/jobs/view/133114754/?trk=jobs_biz_prem_srch,sales manager,are you a dynamic and creative marketing professional looking to make a significant impact in the world of freight and logistics we are seeking a talented marketing manager to ...


In [41]:
empty_count = (job_df["job_description_clean"].str.len() == 0).sum()
print("Empty cleaned descriptions:", empty_count)


Empty cleaned descriptions: 4


In [42]:
if empty_count > 0:
    display(job_df[job_df["job_description_clean"].str.len() == 0][
        ["job_id", "job_title", "location", "job_description"]
    ].head(5))


,job_id,job_title,location,job_description
1187,3693046849,Client Care Coordinator,"Fort Mill, SC",https://housemaster-lake-wylie.careerplug.com/jobs/1870862/apps/new
2184,3693051343,Career Coach,"Long Beach, CA",https://cclbcorps.isolvedhire.com/jobs/945063.html
6252,3694104501,Senior Universal Banker,Nashville Metropolitan Area,
14003,3701312382,Route Sales Representative,Atlanta Metropolitan Area,https://www.paycomonline.net/v4/ats/web.php/jobs/ViewJobDetails?job=121567&clientkey=BE967730582EE9E8D134EF27ED56CCD7&jpt=ac357ef2a85f274d662d993e0b206147


In [46]:
short_count = (job_df["job_description_clean"].str.len() <= 50).sum()
print("Short descriptions (≤50 chars):", short_count)


Short descriptions (≤50 chars): 0


In [45]:
# ==============================
# FINAL TEXT QUALITY FILTER
# ==============================
before = len(job_df)
job_df = job_df[job_df["job_description_clean"].str.len() > 50].copy()
print(f"Dropped {(before - len(job_df))} postings with empty or too-short descriptions (<50 chars)")


Dropped 0 postings with empty or too-short descriptions (<50 chars)


In [47]:
job_df.shape

(15851, 15)

In [49]:
# ==============================
# STEP 9 — Save cleaned jobs
# ==============================
out_path = PROC_DIR + r"\job_postings_cleaned.csv"
job_df.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: D:\Projects\ResumeJobRecommender\data\processed\job_postings_cleaned.csv
